In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "pass123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'grazioso_logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Dashboard layout with the Grazioso Salvare logo and unique identifier
app.layout = html.Div([
    # Header with Grazioso Salvare logo and unique identifier
    html.Center(html.B(html.H1('CS-340 Dashboard - Sean Born'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image))),
    html.Hr(),

    # Filtering options for the dashboard
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Tracking'}
            ],
            value='Water Rescue',  # Default value
            clearable=False,
        ),
    ], style={'width': '50%', 'padding': '20px'}),

    # Interactive data table
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'height': '300px', 'overflowY': 'auto'},
                         page_size=10,  # Pagination for better user experience
                         filter_action='native',  # Enable filtering of columns
                         sort_action='native',  # Enable sorting by columns
                         ),

    html.Hr(),
    
    # Interactive charts layout (Geolocation and Pie chart)
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    filtered_df = df[df['rescue_type'] == filter_type]  # Apply MongoDB filter logic here
    return filtered_df.to_dict('records')

# Pie chart callback to display preferred dog breeds based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Dog Breeds')
    return [dcc.Graph(figure=fig)]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_row):
    if viewData is None or selected_row is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_row[0]  # Only allow single row selection
    lat, lon = dff.iloc[row, 13], dff.iloc[row, 14]  # Adjust column indices for latitude and longitude
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Animal breed
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])  # Animal name
            ])
        ])
    ]

# Run the server
app.run_server(debug=True, port=32900)